<a href="https://colab.research.google.com/github/htried/wikimedia-llm-finetune/blob/main/mpt_7b_instruct_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [ ]:
!pip install --upgrade transformers einops datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00


# Import

In [ ]:
import torch
import transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AdamW
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from typing import Any, Dict, Tuple
import warnings
from threading import Event, Thread
import textwrap
from datasets import concatenate_datasets


import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [ ]:
prompt_template = {
    "description": "A shorter template to experiment with.",
    "prompt_input": "Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    "response_split": "### Response:"
}

class Prompter(object):
    def __init__(self, verbose: bool = False):
        self._verbose = verbose
        self.template = prompt_template
        if self._verbose:
            print(
                f"Using prompt template alpaca: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: str,
        output: str = "",
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        res = self.template["prompt_input"].format(
            instruction=instruction, input=input
        )

        if self._verbose:
            print(res)
        if output != "":
            res = f"{res}{output}"
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()

In [ ]:
model_name = 'htriedman/mpt-7b-instruct-wiki-sparql'

prompter = Prompter()
device_map = "auto"

model = transformers.AutoModelForCausalLM.from_pretrained(
  model_name,
  trust_remote_code=True,
  torch_dtype=torch.bfloat16,
  device_map=device_map,
)
tokenizer = transformers.AutoTokenizer.from_pretrained('mosaicml/mpt-7b-instruct', trust_remote_code=True)

if tokenizer.pad_token_id is None:
    warnings.warn(
        "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
    )
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # "left"
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()
model.to(device=device, dtype=torch.bfloat16)

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- param_init_fns.py
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-instruct:
- modeling_mpt.py
- param_init_fns.py
- custom_embedding.py
- meta_init_context.py
- adapt_tokenizer.py
- attention.py
- blocks.py
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Instantiating an MPTForCausalLM model from /root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/925e0d80e50e77aaddaf9c3ced41ca4ea23a1025/modeling_mpt.py
You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-5-4f8bfc14b2ef>:15: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(50432, 4096)
    (emb_drop): Dropout(p=0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear(in_features=4096, out_features=12288, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (norm_2): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear(in_features=4096, out_features=16384, bias=False)
          (act): GELU(approximate='none')
          (down_proj): Linear(in_features=16384, out_features=4096, bias=False)
        )
        (resid_attn_dropout): Dropout(p=0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
)

In [ ]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

def process_stream(instruction, input, temperature, top_p, top_k, max_new_tokens):
    # Tokenize the input
    input_ids = tokenizer(
        prompter.generate_prompt(instruction, input),
        return_tensors="pt"
    ).input_ids
    input_ids = input_ids.to(model.device)

    # Initialize the streamer and stopping criteria
    streamer = TextIteratorStreamer(
        tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True
    )
    stop = StopOnTokens()

    if temperature < 0.1:
        temperature = 0.0
        do_sample = False
    else:
        do_sample = True

    gkw = {
        "input_ids": input_ids,
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "do_sample": do_sample,
        "top_p": top_p,
        "top_k": top_k,
        "streamer": streamer,
        "stopping_criteria": StoppingCriteriaList([stop]),
    }

    response = ''

    def generate_and_signal_complete():
        model.generate(**gkw)

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    for new_text in streamer:
        response += new_text

    return response

def generate_text(instruction, input, temperature = 0.3, top_p = 0.95, top_k = 0, max_new_tokens = 2000):
    response = process_stream(instruction, input, temperature, top_p, top_k, max_new_tokens)
    return response

In [ ]:
def predict(model, tokenizer, instruction, input):
    s = prompter.generate_prompt(
        instruction,
        input,
    )
    input_ids = tokenizer(s, return_tensors="pt").input_ids
    input_ids = input_ids.to(model.device)
    with torch.no_grad():
        output_ids = model.generate(input_ids, max_new_tokens=150, do_sample=False, top_k=5, top_p=0.95)
    # Slice the output_ids tensor to get only new tokens
    new_tokens = output_ids[0, len(input_ids[0]) :]
    return tokenizer.decode(new_tokens, skip_special_tokens=False)

In [ ]:
ds = load_dataset('htriedman/wiki-sparql', split='validation')

In [ ]:
import gc

In [ ]:
for i in range(3500,3600,10):
    # empty cache
    torch.cuda.empty_cache()
    gc.collect()
    print(f"***PROMPT***:\n\n{ds[i]['input']}\n\n")
    print(f"**EXPECTED**:\n\n{ds[i]['output']}\n\n")
    print(f"***ACTUAL***:\n\n{predict(model, tokenizer, ds[i]['instruction'], ds[i]['input'])}")

***PROMPT***:

Write a SparQL query to count the number of items with P550 (chivalric order) by the number of statements.
Wikidata items:
[]
Wikidata properties:
[['P550', 'chivalric order']]


**EXPECTED**:

#title:Count of items with P550 by number of statements
#defaultView:AreaChart
SELECT ?st ?ct {
  {
    BIND (0 AS ?ct)
    BIND (0 AS ?st)
  }
  UNION {
    SELECT ?st (COUNT(*) as ?ct) {
      ?item wdt:P550 ?value ; wikibase:statements ?st
    }
    GROUP BY ?st ORDER BY ?st
  }
}


***ACTUAL***:

#title:Count of items with P550 by number of statements
#defaultView:AreaChart
SELECT?st?ct {
  {
    BIND (0 AS?ct)
    BIND (0 AS?st)
  }
  UNION {
    SELECT?st (COUNT(*) as?ct) {
     ?item wdt:P550?value ; wikibase:statements?st
    }
    GROUP BY?st ORDER BY?st
  }
}<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoftext|>###<|endoft

In [ ]:
test0 = """
Retrieve all items representing special education schools located in the United States of America.
Wikidata items: [['Q30', 'United States of America'], ['Q1110689', 'special education school']]
Wikidata properties: [['P131', 'located in the administrative territorial entity'], ['P17', 'country'], ['P31', 'instance of']]
"""

test1 = """
Get a list in alphabetical order of all Gen X female politicians in the US
Wikidata items: [['Q30', 'United States of America'], ['Q82955', 'politician'], ['Q6581072', 'female']]
Wikidata properties: [['P27', 'country of citizenship'], ['P106', 'occupation'], ['P21', 'sex or gender'], ['P569', 'date of birth']]
"""

test2 = """
get an alphabetically ordered list of all of the names of democratic mayors of new york city
wikidata items: [['Q82955', 'politician'], ['Q785304', 'Mayor of New York City'], ['Q29552', 'Democratic Party']]
wikidata properties: [['P106', 'occupation'], ['P39', 'position held'], ['P102', 'member of political party']]
"""

In [ ]:
for t in [test0, test1, test2]:
    print(f"***PROMPT***:\n{t}\n")
    print(f"***ACTUAL***:\n")
    print(generate_text(ds[i]['instruction'], t))
    print("===========\n")

***PROMPT***:

Retrieve all items representing special education schools located in the United States of America.
Wikidata items: [['Q30', 'United States of America'], ['Q1110689', 'special education school']]
Wikidata properties: [['P131', 'located in the administrative territorial entity'], ['P17', 'country'], ['P31', 'instance of']]



***ACTUAL***:

SELECT?item WHERE {   item wdt:P17 wd:Q30.   item wdt:P31 wd:Q30.   item wdt:P131 wd:Q1110689.
item (wdt:P17)* wd:Q30. }
None
***PROMPT***:

Get a list in alphabetical order of all Gen X female politicians in the US
Wikidata items: [['Q30', 'United States of America'], ['Q82955', 'politician'], ['Q6581072', 'female']]
Wikidata properties: [['P27', 'country of citizenship'], ['P106', 'occupation'], ['P21', 'sex or gender'], ['P569', 'date of birth']]



***ACTUAL***:

SELECT?item WHERE {   item wdt:P106 wd:Q6581072.   item wdt:P27 wd:Q30.   item wdt:P21 wd:Q6581072.
item wdt:P569?dob.   FILTER( MONTH(?dob) = 1 && DAY(?dod) < 32 ).   item